
Driven Data Water Contest

Import needed libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#import data
train = pd.read_csv('/Users/ericp/OneDrive/Documents/GitHub/datadrivenH2O/train.csv')
target = pd.read_csv('/Users/ericp/OneDrive/Documents/GitHub/datadrivenH2O/target.csv')
test = pd.read_csv('/Users/ericp/OneDrive/Documents/GitHub/datadrivenH2O/test.csv')
train_id = train['id']
test_id = test['id']
train = train.drop(['id'], axis = 1)
test = test.drop(['id'], axis = 1)
colnames = train.columns
train_shape = train.shape[0]
train.head(10)


In [ ]:
#look at train / test shapes
print(train.shape)
print(test.shape)
print(target.shape)

In [ ]:
#look at the value_counts of the target variable
target['status_group'].value_counts()

In [ ]:
#LabelEncode target variable
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
target['status_group'] = le.fit_transform(target['status_group'])
target['status_group'].value_counts()
#functional = 0
#non-functional = 1
#functional needs repair = 2

In [ ]:
#look at correlations of variables with target
#check for correlation between target and predictors
target_corr = list()

for c, v in enumerate(train, start = 1):
    target_corr.append(target.corrwith(train[v], method = 'spearman'))
    
target_corr = pd.Series(data = target_corr, index = train.columns, name = 'correlation')
target_corr = abs(target_corr)
target_corr
#not a lot highly correlated with the target

In [ ]:
#combine data
combine = pd.concat([train, test],  axis = 0).reset_index(drop = True)

In [ ]:
#look for missing values
miss_vals = pd.Series(combine.isnull().sum(), name = 'PctMissing')
miss_vals = miss_vals[miss_vals!=0]
miss_vals = miss_vals.sort_values(ascending = False)
print(miss_vals)

In [ ]:
#pct missing
miss_vals / len (combine)

In [ ]:
#these variables all seem to be about area / region. Might be best to use the mode of the region they're in
combine['subvillage'] = combine.groupby('region')['subvillage'].transform(lambda x:x.fillna(x.mode()[0]))
combine['public_meeting'] = combine.groupby('region')['public_meeting'].transform(lambda x:x.fillna(x.mode()[0]))
combine['permit'] = combine.groupby('region')['permit'].transform(lambda x:x.fillna(x.mode()[0]))
combine['funder'] = combine.groupby('region')['funder'].transform(lambda x:x.fillna(x.mode()[0]))
combine['installer'] = combine.groupby('region')['funder'].transform(lambda x:x.fillna(x.mode()[0]))
combine['scheme_management'] = combine.groupby('region')['scheme_management'].transform(lambda x:x.fillna(x.mode()[0]))

In [ ]:
#scheme_management and scheme_name seem to be redundant. Lots of missing values for scheme_name. Will delete.
#region and region_code same. Will delete region
#extraction_type_group seems identical to extraction_type
combine = combine.drop(['scheme_name', 'region', 'extraction_type_group', 'management_group', 'payment_type'], axis = 1)

In [ ]:
#check missing values
miss_vals = pd.Series(combine.isnull().sum(), name = 'PctMissing')
miss_vals = miss_vals[miss_vals!=0]
miss_vals = miss_vals.sort_values(ascending = False)
print(miss_vals)

In [ ]:
dtyp = pd.Series(combine.dtypes, name = 'dtype')
dtyp

In [ ]:
#some items as int are actually objects
combine['construction_year'] = combine['construction_year'].astype('object')

In [ ]:
#check correlations between variables. See if there's some that can be deleted
f = plt.figure(figsize=(8, 8))
plt.matshow(combine.corr(), fignum=f.number)
plt.yticks(range(combine.select_dtypes(['number']).shape[1]), combine.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
corr_df = combine.corr()
corr_df
#all correlations quite low. Keep all predictors.

In [ ]:
#label encode columns where there is structure (i.e. levels)
le_columns = ['date_recorded', 'permit']

In [ ]:
train = combine[:train_shape]
test = combine[train_shape:]
print(train.shape)
print(test.shape)

In [ ]:
#import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
#create metric to determine accuracy
from sklearn.model_selection import KFold, cross_val_score
def accuracy(model, X, y, n = 5):
    kf = KFold(n, random_state = 1, shuffle = True)
    acc = cross_val_score(model, X, y, scoring = 'accuracy', cv = kf)
    return acc    

Create Base Models

In [ ]:
#create base models
lgb = LGBMClassifier()
rf = RandomForestClassifier()
xgb = XGBClassifier()

Base Model Scores

In [ ]:
score = accuracy(rf, train, target)